<a href="https://colab.research.google.com/github/emrealtinok/neural_style_transfer/blob/main/NeuralStyleTransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import vgg19


base_image_path = tf.keras.utils.get_file(
    "dali.jpg", "https://wallpaperaccess.com/full/300219.jpg"
)

style_reference_image_path = tf.keras.utils.get_file(
    "vangogh", "https://img.rawpixel.com/s3fs-private/rawpixel_images/website_content/pdvangogh-dt1567.jpg?w=1000&dpr=1&fit=default&crop=default&q=65&vib=3&con=3&usm=15&bg=F4F4F3&ixlib=js-2.2.1&s=542c62759cf4dc748b9d7df513560ee0"
)

style_reference_image_path_2 = tf.keras.utils.get_file(
    "picasso.jpg", "https://www.itl.cat/pngfile/big/5-57917_pablo-picasso-paintings-wide-hd-wallpaper-modern-art.jpg"
)

result_prefix = "dali_generated"

# Weights of the different loss components
total_variation_weight = 1e-11
style_weight = 1e-5
content_weight = 2.5e-8

# Dimensions of the generated picture
width, height = tf.keras.preprocessing.image.load_img(base_image_path).size
img_nrows = 720
img_ncols = int(width * img_nrows / height)

In [2]:
from IPython.display import Image, display

display(Image(base_image_path))
display(Image(style_reference_image_path))
display(Image(style_reference_image_path_2))


In [3]:
def preprocess_image(image_path):
    # Util function to open, resize and format pictures into appropriate tensors
    img = tf.keras.preprocessing.image.load_img(
        image_path, target_size=(img_nrows, img_ncols)
    )
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return tf.convert_to_tensor(img)

def deprocess_image(x):
    # Util function to convert a tensor into a valid image
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype("uint8")
    return x

In [4]:
# The gram matrix of an image tensor (feature-wise outer product)


def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram


# The "style loss" is designed to maintain
# the style of the reference image in the generated image.
# It is based on the gram matrices (which capture style) of
# feature maps from the style reference images
# and from the generated image


def style_loss(style, style_2, combination):
    S = gram_matrix(style)
    S2 = gram_matrix(style_2)
    S = (S + S2) / 2
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return tf.reduce_sum(tf.square(S - C)) / (4 * (channels ** 2) * (size ** 2))


# An auxiliary loss function
# designed to maintain the "content" of the
# base image in the generated image


def content_loss(base, combination):
    return tf.reduce_sum(tf.square(combination - base))


# The 3rd loss function, total variation loss,
# designed to keep the generated image locally coherent


def total_variation_loss(x):
    a = tf.square(
        x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, 1:, : img_ncols - 1, :]
    )
    b = tf.square(
        x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, : img_nrows - 1, 1:, :]
    )
    return tf.reduce_sum(tf.pow(a + b, 1.25))

In [5]:
# Builds a VGG19 model loaded with pre-trained ImageNet weights
model = vgg19.VGG19(weights="imagenet", include_top=False)

# Gets the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# Sets up a model that returns the activation values for every layer in
# VGG19 (as a dict).
feature_extractor = tf.keras.Model(inputs=model.inputs, outputs=outputs_dict)

In [ ]:
# Layer names
print(outputs_dict.keys())

In [7]:
# List of layers to use for the style loss.
style_layer_names = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1"
]
# The layer to use for the content loss
content_layer_name = "block5_conv2"

# Weights of each style layer
style_weights = [1, 1, 1, 1, 1]
individual_style_weights = dict([(style_layer_names[i], style_weights[i]) for i in range(len(style_layer_names))])

def compute_loss(combination_image, base_image, style_reference_image, style_reference_image_2):
    input_tensor = tf.concat(
        [base_image, style_reference_image, style_reference_image_2, combination_image], axis=0
    )
    features = feature_extractor(input_tensor)

    # Initialize the loss
    loss = tf.zeros(shape=())

    # Add content loss
    layer_features = features[content_layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[3, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )

    # Add style loss
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        style_reference_features_2 = layer_features[2, :, :, :]
        combination_features = layer_features[3, :, :, :]

        # Adds style loss for both style images
        sl = style_loss(style_reference_features, style_reference_features_2, combination_features)
        loss += (style_weight / (len(style_layer_names))) * sl * individual_style_weights[layer_name]
 


    # Add total variation loss
    loss += total_variation_weight * total_variation_loss(combination_image)
    return loss

In [8]:
# Defines function that computes the loss and gradients
@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image, style_reference_image_2):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image, style_reference_image_2)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

In [ ]:
# Optimizer
optimizer = tf.keras.optimizers.SGD(
    tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=250, decay_rate=0.96
    )
)

# Preprocessing
base_image = preprocess_image(base_image_path)
style_reference_image = preprocess_image(style_reference_image_path)
style_reference_image_2 = preprocess_image(style_reference_image_path_2)
combination_image = tf.Variable(preprocess_image(base_image_path))

#Applies the style transfer
iterations = 20000
for i in range(1, iterations + 1):
    loss, grads = compute_loss_and_grads(
        combination_image, base_image, style_reference_image, style_reference_image_2)
    optimizer.apply_gradients([(grads, combination_image)])
    if i % 1000 == 0:
        print("Iteration %d: loss=%.2f" % (i, loss))
        img = deprocess_image(combination_image.numpy())
        fname = result_prefix + "_at_iteration_%d.png" % i
        tf.keras.preprocessing.image.save_img(fname, img)

In [ ]:
display(Image(result_prefix + "_at_iteration_20000.png"))